# Volatility

In [ ]:
# %pip install pandas scipy requests matplotlib seaborn

In [ ]:
import re
import time
from time import sleep

import requests
import math
import pandas as pd
from scipy.stats import norm

In [ ]:
API_KEY = {'X-API-Key': '114514'}
BASE_URL = "http://localhost:9999/v1"

DEBUG = True
CHECK_TIME = False
CHECK_DATAFRAME = True

RISK_EXPOSURE = 7000
DELTA_HEDGE_QUANTITY = 3000
RUN_STRATEGY = True

MAX_OPTION_ORDER_SIZE = 5
SPEED_BUMP = 0.2

In [ ]:
def bs(S, K, T, r, sigma, option_type):
    d1 = calculate_d1(sigma, S, K, T)
    d2 = calculate_d2(d1, sigma, T)

    if option_type == 'call':
        return S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        return K * math.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)


def calculate_d1(sigma, S, K, T):
    try:
        return (1 / (sigma * math.sqrt(T))) * (math.log(S / K) + (sigma ** 2 / 2) * T)
    except ZeroDivisionError:
        print(sigma, " ", S, " ", K, " ", T)


def calculate_d2(d1, sigma, T):
    return d1 - sigma * math.sqrt(T)


def implied_vol(S, K, T, r, price, option_type):
    vol = 0.22  # start value for the volatility
    d1 = calculate_d1(vol, S, K, T)
    test_price = bs(S, K, T, r, vol, option_type)  # BS price using the testing vol
    vega = S * math.sqrt(T) * norm.cdf(d1)  # vega of the option
    i = 0

    while True:
        vol = vol + (price - test_price) / vega
        if vol < 0.00001:
            vol = 0.00001
        d1 = calculate_d1(vol, S, K, T)
        vega = S * math.sqrt(T) * norm.cdf(d1)
        test_price = bs(S, K, T, r, vol, option_type)  # BS price using the testing vol

        diff = abs(price - test_price)
        i = i + 1

        if diff < abs(0.000001) or i > 200 or vol == 0.00001:
            break

    return vol


def delta_from_option(d1, position, option_type):
    if option_type == 'call':
        return norm.cdf(d1) * position * 100
    elif option_type == 'put':
        return (norm.cdf(d1) - 1) * position * 100
    else:
        raise Exception('Wrong option type')


def calculate_all_data(tick, df, sec, realized_vol, stock_last_price, option_time_to_maturity):
    df['tick'] = tick
    df['d1'] = df.apply(
        lambda row: calculate_d1(realized_vol, stock_last_price, row['strike'], option_time_to_maturity), axis=1)
    df['d2'] = df.apply(lambda row: calculate_d2(row['d1'], realized_vol, option_time_to_maturity), axis=1)

    # theoretical price
    df['thp'] = df.apply(
        lambda row: bs(stock_last_price, row['strike'], option_time_to_maturity, 0, realized_vol, row['type']), axis=1)
    df['thp'] = df['thp'].apply(lambda x: round(x, 2))

    # extract securities info
    df['last'] = sec['last'].tail(10).reset_index(drop=True)
    df['position'] = sec['position'].tail(10).reset_index(drop=True)

    # implied vol
    df['iv'] = df.apply(
        lambda row: implied_vol(stock_last_price, row['strike'], option_time_to_maturity, 0, row['last'], row['type']),
        axis=1)
    # df['iv'] = df['iv'].apply(lambda x: round(x, 4))

    # d1 calculated with implied vol
    df['d1_iv'] = df.apply(
        lambda row: calculate_d1(row['iv'], stock_last_price, row['strike'], option_time_to_maturity), axis=1)

    # delta
    df['delta'] = df.apply(lambda row: delta_from_option(row['d1_iv'], row['position'], row['type']), axis=1)

    df['vega'] = df.apply(
        lambda row: row['strike'] * math.sqrt(option_time_to_maturity) * norm.cdf(abs(row['d1'])), axis=1)


In [ ]:
def init_data():
    df = pd.DataFrame(columns=['name', 'type', 'strike'])
    df['name'] = ['RTM48C', 'RTM48P', 'RTM49C', 'RTM49P', 'RTM50C', 'RTM50P', 'RTM51C', 'RTM51P', 'RTM52C', 'RTM52P']
    df['type'] = ['call', 'put'] * 5
    df['strike'] = [48, 48, 49, 49, 50, 50, 51, 51, 52, 52]

    return df


def parse_news(session):
    news = session.get(BASE_URL + '/news').json()

    if len(news) == 0:
        return 0

    if len(news) < 3:
        return int(re.findall(r'\d+', news[-1]['body'])[1]) / 100

    if len(news) % 2 == 1:
        newest_news = news[0]
    else:
        newest_news = news[1]

    return float(newest_news['body'][-3:-1]) / 100


def delta_hedge_by_stock(session, df, sec):
    delta = df['delta'].sum() + sec.at[0, 'position']

    if delta >= DELTA_HEDGE_QUANTITY:
        post_order(session, 'MARKET', DELTA_HEDGE_QUANTITY, 'SELL', 'RTM')
    elif delta <= -DELTA_HEDGE_QUANTITY:
        post_order(session, 'MARKET', DELTA_HEDGE_QUANTITY, 'BUY', 'RTM')


def post_order(s, type, quantity, action, ticker):
    r = s.post("http://localhost:9999/v1/orders?type="
            + type + "&quantity=" + str(quantity) + "&action=" + action + "&ticker=" + ticker)

    if r.status_code == 429:
        wait = r.json()["wait"]
        sleep(wait)

        post_order(s, type, quantity, action, ticker)
    elif r.status_code != 200:
        print(r.json())


def straddle(s, order_type, price, quantity, action, ticker):
    for _ in range(5):
        post_order(s, order_type, quantity, action, ticker + str(price) + 'C')
        post_order(s, order_type, quantity, action, ticker + str(price) + 'P')




def record_dataframe_per_second(session):
    time_remaining = 600
    # recorded_iv = parse_news(session)
    # inverse_relized_vol_time = 0
    df = init_data()
    recorded_df = []
    
    prev_tick = session.get(BASE_URL + '/case').json()['tick']

    while 0 < prev_tick < 600:
        # base info
        tick = session.get(BASE_URL + '/case').json()['tick']

        if tick == prev_tick:
            continue

        if tick == 1:
            continue

        time_remaining = 600 - tick
        days_remaining = math.ceil(time_remaining / 30)
        option_time_to_maturity = days_remaining / 240

        realized_vol = parse_news(session)

        # calculate all data
        sec = pd.DataFrame(session.get(BASE_URL + '/securities').json())
        stock_last_price = sec.at[0, 'last']
        calculate_all_data(tick, df, sec, realized_vol, stock_last_price, option_time_to_maturity)

        # append to recorded_df
        recorded_df.append(df.copy())

        # delta_hedge_by_stock(session, df, sec)

        # print("total delta: ", df['delta'].sum() + sec.at[0, 'position'])
        # print("sec position: ", sec.at[0, 'position'])

        prev_tick = tick

    return recorded_df


In [ ]:
def start_record():
    with requests.Session() as session:
        session.headers.update(API_KEY)
        r = session.get(BASE_URL + '/case')

        recorded = record_dataframe_per_second(session)
        df = pd.concat(recorded)
        df.to_csv('recorded_per_second.csv')
        print('done')

# start_record()

In [ ]:
def cal_rsi(df, n=14):
    df['delta'] = df['iv'].diff()
    df['gain'] = df['delta'].apply(lambda x: x if x > 0 else 0)
    df['loss'] = df['delta'].apply(lambda x: -x if x < 0 else 0)
    df['avg_gain'] = df['gain'].rolling(n).mean()
    df['avg_loss'] = df['loss'].rolling(n).mean()
    df['rs'] = df['avg_gain'] / df['avg_loss']
    df['rsi'] = 100 - (100 / (1 + df['rs']))
    df = df.drop(columns=['delta', 'gain', 'loss', 'avg_gain', 'avg_loss', 'rs'])
    rsi = df['rsi'].tail(1).values[0]

    return rsi


def momemtum_strategy(session, df, sec):
    '''
    1. pick the price with highest vega
    2. pick from put and call, which current price is higher
    3. speculate on this put/call's iv.
    '''

    """
    if current_position is 0, pick the price with highest vega
    else, choose the price with current position
    """
    sec = session.get(BASE_URL + '/securities').json()

    holding_position = True

    if not holding_position:
        last_price = sec[0]['last']
        if last_price >= 50: 
            selected_strike_price = 48
            index = 1
        else:
            selected_strike_price = 52
            index = 9
        
        call_last_price = sec[index]['last']
        put_last_price = sec[index + 1]['last']
        if call_last_price > put_last_price:
            selected_type = 'C'
        else:
            selected_type = 'P'
    else:
        # find current position
        raise NotImplementedError

    emas = df[df['name'] == 'RTM' + str(selected_strike_price) + selected_type]['iv'].tail(1).values[0]
    emal = df[df['name'] == 'RTM' + str(selected_strike_price) + selected_type]['iv'].tail(5).mean()

    rsi = cal_rsi(df[df['name'] == 'RTM' + str(selected_strike_price) + selected_type], 10)

    # speculation:
    # if emas > emal and rsi > 60, then buy straddle
    # elif emas < emal and rsi < 40, then sell straddle
    if emas > emal and rsi > 60:
        straddle(session, 'MARKET', selected_strike_price, 100, 'BUY', 'RTM')
    elif emas < emal and rsi < 40:
        straddle(session, 'MARKET', selected_strike_price, 100, 'SELL', 'RTM')


def option_algo_trading():
    with requests.Session() as session:
        session.headers.update(API_KEY)
        r = session.get(BASE_URL + '/case')
        if r.status_code != 200:
            print(r.json())
            raise Exception('Failed to get case')
        
        time_remaining = 600
        total_df = init_data()
        prev_tick = r.json()['tick']

        while 0 < prev_tick < 600:
            tick = session.get(BASE_URL + '/case').json()['tick']

            if tick == 1:
                continue

            time_remaining = 600 - tick
            days_remaining = math.ceil(time_remaining / 30)
            option_time_to_maturity = days_remaining / 240

            df = init_data()
            realized_vol = parse_news(session)
            sec = pd.DataFrame(session.get(BASE_URL + '/securities').json())
            stock_last_price = sec.at[0, 'last']
            calculate_all_data(tick, df, sec, realized_vol, stock_last_price, option_time_to_maturity)

            if tick != prev_tick:
                total_df = pd.concat([total_df, df])
                momemtum_strategy(session, total_df, sec)
            
            # here we can improve the strategy (let delta run?)
            # if we long straddle, then we can hedge when exceed exposeure
            delta_hedge_by_stock(session, df, sec)

            prev_tick = tick

option_algo_trading()

record sigma of underlying asset, use for indicators?

actually, we're trading vega.

## Momentum indicators

#### MACD

#### RSI

#### KDJ

